## [作業重點]
確保你了解隨機森林模型中每個超參數的意義，並觀察調整超參數對結果的影響

## 作業

1. 試著調整 RandomForestClassifier(...) 中的參數，並觀察是否會改變結果？
2. 改用其他資料集 (boston, wine)，並與回歸模型與決策樹的結果進行比較

In [1]:
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
# 如果是分類問題，請使用 DecisionTreeClassifier，若為回歸問題，請使用 DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor



关于参数
其中关于决策树的参数：

criterion: ”gini” or “entropy”(default=”gini”)是计算属性的gini(基尼不纯度)还是entropy(信息增益)，来选择最合适的节点。
splitter: ”best” or “random”(default=”best”)随机选择属性还是选择不纯度最大的属性，建议用默认。
max_features: 选择最适属性时划分的特征不能超过此值。

当为整数时，即最大特征数；当为小数时，训练集特征数*小数；
#if “auto”, then max_features=sqrt(n_features).
#If “sqrt”, thenmax_features=sqrt(n_features).
#If “log2”, thenmax_features=log2(n_features).
#If None, then max_features=n_features.
#max_depth: (default=None)设置树的最大深度，默认为None，这样建树时，会使每一个叶节点只有一个类别，或是达到min_samples_split。
#min_samples_split:根据属性划分节点时，每个划分最少的样本数。
#min_samples_leaf:叶子节点最少的样本数。
#max_leaf_nodes: (default=None)叶子树的最大样本数。
#min_weight_fraction_leaf: (default=0) 叶子节点所需要的最小权值
verbose:(default=0) 是否显示任务进程

关于随机森林特有的参数：
n_estimators=10：决策树的个数，越多越好，但是性能就会越差，至少100左右（具体数字忘记从哪里来的了）可以达到可接受的性能和误差率。 
bootstrap=True：是否有放回的采样。
  
oob_score=False：oob（out of band，带外）数据，即：在某次决策树训练中没有被bootstrap选中的数据。多单个模型的参数训练，我们知道可以用cross validation（cv）来进行，但是特别消耗时间，而且对于随机森林这种情况也没有大的必要，所以就用这个数据对决策树模型进行验证，算是一个简单的交叉验证。性能消耗小，但是效果不错。
  
n_jobs=1：并行job个数。这个在ensemble算法中非常重要，尤其是bagging（而非boosting，因为boosting的每次迭代之间有影响，所以很难进行并行化），因为可以并行从而提高性能。1=不并行；n：n个并行；-1：CPU有多少core，就启动多少job
warm_start=False：热启动，决定是否使用上次调用该类的结果然后增加新的。  
class_weight=None：各个label的权重。


https://blog.csdn.net/u012102306/article/details/52228516

In [2]:
wine = datasets.load_wine()

x_train,x_test,y_train,y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=4)

# 建立模型 (使用 20 顆樹，每棵樹的最大深度為 5)
clf = RandomForestClassifier(n_estimators=20, criterion='entropy',  max_depth=5, oob_score=False)

# 訓練模型
clf.fit(x_train, y_train)

# 預測測試集
y_pred = clf.predict(x_test)

In [3]:
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
R2_score = r2_score(y_test, y_pred)
print("R2 score : %f "%R2_score)

R2 score : 1.000000 


In [5]:
print(wine.feature_names)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [13]:
import pandas as pd
# 查看欄位重要性
pd.concat( (pd.DataFrame(wine.feature_names, columns=['variable']),
           pd.DataFrame(clf.feature_importances_, columns=['importance'])),
         axis=1).sort_values(by='importance', ascending=False)

,variable,importance
6,flavanoids,0.211978
12,proline,0.155829
9,color_intensity,0.153754
10,hue,0.109381
0,alcohol,0.096156
11,od280/od315_of_diluted_wines,0.086827
5,total_phenols,0.039428
1,malic_acid,0.031545
3,alcalinity_of_ash,0.031224
4,magnesium,0.026243


DecisionTree

In [8]:
# 建立模型
DTC = DecisionTreeClassifier(max_depth=8 ,min_samples_split=10, min_samples_leaf=3)
# 訓練模型
DTC.fit(x_train,y_train)
# 預測測試集
y_pred = DTC.predict(x_test)

In [9]:
R2_score = r2_score(y_test, y_pred)
print("R2 score : %f "%R2_score)

R2 score : 0.766667 


LinearRegression

In [11]:
from sklearn.linear_model import LinearRegression
# 建立模型
lr = LinearRegression()

# 訓練模型
lr.fit(x_train, y_train)

# 預測測試集
y_pred = lr.predict(x_test)

In [12]:
R2_score = r2_score(y_test, y_pred)
print("R2 score : %f "%R2_score)

R2 score : 0.902314 
